# LangGraph Tutorial: Building a Conditional Graph with Tools

In this tutorial, we'll explore **LangGraph** using a `StateGraph`. We'll cover:
1. Creating nodes and edges
2. Using routers for conditional branching
3. Adding and using tools
4. Executing the graph with different states

In [1]:
# Install langgraph if you haven't already
!pip install langgraph typing-extensions

In [2]:
# Imports
from langgraph.graph import START, StateGraph, END
from typing_extensions import TypedDict

## 1️⃣ Define the State
We'll define a simple state that just contains a `text` field.

In [3]:
class State(TypedDict):
    text: str

## 2️⃣ Define Nodes
Nodes are functions that take the state and return a new state.

In [4]:
def node_a(state: State) -> dict:
    return {"text": state["text"] + "a"}

def node_b(state: State) -> dict:
    return {"text": state["text"] + "b"}

def node_c(state: State) -> dict:
    return {"text": state["text"] + "c"}

## 3️⃣ Define a Router
The router decides which node to go to next based on the state.

In [5]:
def router(state: State) -> str:
    """If the text contains 'b' and length < 5, go to node_c; otherwise end."""
    if "b" in state["text"] and len(state["text"]) < 5:
        return "node_c"
    else:
        return END

## 4️⃣ Define a Tool Node
Tools are just nodes that perform some extra action. Here, we create a tool that **reverses the text**.

In [6]:
def reverse_tool(state: State) -> dict:
    return {"text": state["text"][::-1]}

## 5️⃣ Build the Graph

In [7]:
graph = StateGraph(State)

# Add nodes
graph.add_node("node_a", node_a)
graph.add_node("node_b", node_b)
graph.add_node("node_c", node_c)
graph.add_node("reverse_node", reverse_tool)

# Add edges
graph.add_edge(START, "node_a")
graph.add_edge("node_a", "node_b")

# Conditional edge using router
graph.add_conditional_edges("node_b", router)

# If node_c runs, we optionally reverse before ending
graph.add_edge("node_c", "reverse_node")
graph.add_edge("reverse_node", END)

# Compile the graph
app = graph.compile()

## 6️⃣ Run the Graph

In [8]:
print("Executing with initial text: ''")
print(app.invoke({"text": ""}))  # Should follow a -> b -> c -> reverse -> END

print("\nExecuting with initial text: 'long_text_'")
print(app.invoke({"text": "long_text_"}))  # Should follow a -> b -> END

Executing with initial text: ''
{'text': 'cba'}

Executing with initial text: 'long_text_'
{'text': 'long_text_ab'}


✅ **Congratulations!**

You now have a complete LangGraph workflow with:
- Regular nodes
- Conditional branching via a router
- A tool node
- Dynamic execution depending on the initial state